<a href="https://colab.research.google.com/github/abhishekkm8088/Pyspark-Abhi/blob/main/ScalaAdvProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing the scala environment

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!wget -q https://github.com/scala/scala/releases/download/v2.12.18/scala-2.12.18.deb
!tar xf spark-3.4.1-bin-hadoop3.tgz
!dpkg -i scala-2.12.18.deb

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

!curl -Lo coursier https://git.io/coursier-cli && chmod +x coursier
!./coursier launch --fork almond --scala 2.12.10 -- --install

(Reading database ... 130340 files and directories currently installed.)
Preparing to unpack scala-2.12.18.deb ...
Unpacking scala (2.12.18-400) over (2.12.18-400) ...
Setting up scala (2.12.18-400) ...
Processing triggers for man-db (2.10.2-1) ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42577  100 42577    0     0  78905      0 --:--:-- --:--:-- --:--:-- 78905
Error: /root/.local/share/jupyter/kernels/scala already exists, pass --force to force erasing it


In [ ]:
!ls /usr/lib/jvm/java-11-openjdk-amd64
!ls /content/spark-3.4.1-bin-hadoop3
!ls /content/scala-2.12.18.deb
!echo $JAVA_HOME
!echo $SPARK_HOME

bin  conf  docs  include  jmods  legal	lib  man  release
bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin
/content/scala-2.12.18.deb
/usr/lib/jvm/java-11-openjdk-amd64
/content/spark-3.4.1-bin-hadoop3


In [ ]:
!pip install -q findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Colab Scala Spark") \
    .getOrCreate()

spark

In [ ]:
%%writefile HelloWorld.scala
import org.apache.spark.sql.SparkSession

object HelloWorld {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

    // Sample data
    val rdd = sc.parallelize(Seq(
      ("apple", 3), ("apple", 4),
      ("apple", 2), ("banana", 3),
      ("banana", 1), ("orange", 5),
      ("apple", 6)
    ))

    // Using combineByKey (Advanced Transformation)
    val combined = rdd.combineByKey(
      (value: Int) => (value, 1),                                 // CreateCombiner
      (acc1: (Int, Int), value: Int) => (acc1._1 + value, acc1._2 + 1), // MergeValue
      (acc1: (Int, Int), acc2: (Int, Int)) => (acc1._1 + acc2._1, acc1._2 + acc2._2)  // MergeCombiners
    )

    // Compute average per key
    val averages = combined.mapValues { case (sum, count) => sum.toDouble / count }

    averages.collect().foreach(println)

    spark.stop()
  }
}


Overwriting HelloWorld.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld.scala

In [ ]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" HelloWorld

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 04:53:06 INFO SparkContext: Running Spark version 3.4.1
25/08/01 04:53:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 04:53:07 INFO ResourceUtils: ==============================================================
25/08/01 04:53:07 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 04:53:07 INFO ResourceUtils: ==============================================================
25/08/01 04:53:07 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 04:53:07 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 04:53

In [ ]:
%%writefile evenSquares.scala
import org.apache.spark.sql.SparkSession

object evenSquares {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

    // Sample data
     val nums = List(1, 2, 3, 4, 5)

    val evenSquares = nums.filter(_ % 2 == 0).map(x => x * x)

    evenSquares.foreach(println)



    spark.stop()
  }
}


Overwriting evenSquares.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" evenSquares.scala

In [ ]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" evenSquares

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 05:24:49 INFO SparkContext: Running Spark version 3.4.1
25/08/01 05:24:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 05:24:50 INFO ResourceUtils: ==============================================================
25/08/01 05:24:50 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 05:24:50 INFO ResourceUtils: ==============================================================
25/08/01 05:24:50 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 05:24:50 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 05:24

In [ ]:
%%writefile distinct.scala
import org.apache.spark.sql.SparkSession

object distinct {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

    // Sample data
     val numbers = List(1, 2, 2, 3, 4, 4, 5, 6)
     val result = numbers.filter(_ % 2 == 0).map(x => x * x).distinct
     println(result)




    spark.stop()
  }
}

Overwriting distinct.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" distinct.scala

In [ ]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" distinct

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 06:29:52 INFO SparkContext: Running Spark version 3.4.1
25/08/01 06:29:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 06:29:53 INFO ResourceUtils: ==============================================================
25/08/01 06:29:53 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 06:29:53 INFO ResourceUtils: ==============================================================
25/08/01 06:29:53 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 06:29:54 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 06:29

In [ ]:
%%writefile Accumilator.scala
import org.apache.spark.sql.SparkSession

object Accumilator {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

   /////////////////////////////////////////cretae an Accumilator variables

   val errorCount = sc.longAccumulator("Error Count")

   //sample RDD with some log lines
   val data = sc.parallelize(Seq("INFO Start", "ERROR Failure1", "ERROR Failure2", "INFO End"))

   //use the Accumilator
   data.foreach(line => {
     if (line.contains("ERROR")) {
       errorCount.add(1)
     }
   })

   println("Total errors: " + errorCount.value)




    spark.stop()
  }
}

Writing Accumilator.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" Accumilator.scala

In [ ]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" Accumilator

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 06:34:47 INFO SparkContext: Running Spark version 3.4.1
25/08/01 06:34:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 06:34:48 INFO ResourceUtils: ==============================================================
25/08/01 06:34:48 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 06:34:48 INFO ResourceUtils: ==============================================================
25/08/01 06:34:48 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 06:34:48 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 06:34

In [ ]:
%%writefile broadcast.scala
import org.apache.spark.sql.SparkSession

object broadcast {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

   //create a lookup map to broadcast
   val CountryCodes = Map("USA" -> "US", "India" -> "IN", "Germany" -> "DE")
   val lookup = sc.broadcast(CountryCodes)

   //broadcast the variables
   val broadcastCountryCodes = sc.broadcast(CountryCodes)

   //Sample RDD of country codes
   val countries = sc.parallelize("USA", "IN", "Germany")

   //Map using the broadcast variable
   val  fullNames = countries.map(country => (country, lookup.value.getOrElse(country, "Unknown")))






    spark.stop()
  }
}

Overwriting broadcast.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" broadcast.scala

broadcast.scala:21: error: too many arguments (3) for method parallelize: (seq: Seq[T], numSlices: Int)(implicit evidence$1: scala.reflect.ClassTag[T])org.apache.spark.rdd.RDD[T]
   val countries = sc.parallelize("USA", "IN", "Germany")
                                               ^
one error found


In [ ]:
%%writefile Persist.scala
import org.apache.spark.sql.SparkSession

object Persist {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder()
      .appName("ScalaAdvancedTransformations")
      .master("local[*]")
      .config("spark.driver.memory", "1g")  // Increase memory to 1 GB
      .getOrCreate()

    val sc = spark.sparkContext

   val bigData = sc.parallelize(1 to 10000000)

// Persist to memory and spill to disk if needed
val squares = bigData.map(x => x * x).persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK)

// First action triggers persistence
println("First 5: " + squares.take(5).mkString(", "))

// Reuse persisted data
println("Count: " + squares.count())



    spark.stop()
  }
}

Overwriting Persist.scala


In [ ]:
!scalac -classpath "$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" Persist.scala

In [ ]:
!scala -J-Xmx1g -classpath ".:$(echo /content/spark-3.4.1-bin-hadoop3/jars/*.jar | tr ' ' ':')" Persist

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 07:11:09 INFO SparkContext: Running Spark version 3.4.1
25/08/01 07:11:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/01 07:11:09 INFO ResourceUtils: ==============================================================
25/08/01 07:11:09 INFO ResourceUtils: No custom resources configured for spark.driver.
25/08/01 07:11:09 INFO ResourceUtils: ==============================================================
25/08/01 07:11:09 INFO SparkContext: Submitted application: ScalaAdvancedTransformations
25/08/01 07:11:10 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/08/01 07:11